In [1]:
import os
import pandas as pd
import glob

In [2]:
def extract_time(time):
    # 2019-12-20 09-23-47
    return(pd.to_datetime(time.split('.')[0],format='%Y-%m-%d %H-%M-%S'))

In [18]:
# c:\Windows\exiftool(-k).exe" . -r -ext mp4 -api largefilesupport=1 -json > video.json

videos = pd.read_json('../WA2019_V01/video.json')
videos['BaseName'] = videos.SourceFile.apply(os.path.basename)
videos['Station']=videos.BaseName.str.split('_',expand=True)[4]
videos['StartTime']=videos.BaseName.str.split('_',expand=True)[5].apply(extract_time)
videos['EndTime'] = videos['StartTime']+pd.to_timedelta(videos.Duration)
#videos.FileCreateDate.dt.tz_convert()

In [19]:
temp =videos.SourceFile.str.split('/',expand=True)[2]
videos['Station'] =temp.str.split('_',expand=True)[2]

In [20]:
videos

,AudioBitsPerSample,AudioChannels,AudioFormat,AudioSampleRate,AvgBitrate,Balance,BitDepth,CompatibleBrands,CompressorID,CreateDate,...,TrackLayer,TrackModifyDate,TrackVolume,VideoFrameRate,XResolution,YResolution,BaseName,Station,StartTime,EndTime
0,16,2,mp4a,44100,161 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_002_2019-12-20 09-23-47.mp4,002,2019-12-20 09:23:47,2019-12-20 09:43:31
1,16,2,mp4a,44100,148 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_003_2019-12-21 09-06-45.mp4,003,2019-12-21 09:06:45,2019-12-21 09:24:14
2,16,2,mp4a,44100,156 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_005_2019-12-21 08-00-31.mp4,005,2019-12-21 08:00:31,2019-12-21 08:20:40
3,16,2,mp4a,44100,157 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_006_2019-12-20 08-22-35.mp4,006,2019-12-20 08:22:35,2019-12-20 08:43:31
4,16,2,mp4a,44100,176 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_007_2019-12-21 06-56-54.mp4,007,2019-12-21 06:56:54,2019-12-21 07:16:34
5,16,2,mp4a,44100,165 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_010_2019-12-21 05-32-22.mp4,010,2019-12-21 05:32:22,2019-12-21 05:56:20
6,16,2,mp4a,44100,150 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_011_2019-12-19 08-36-06.mp4,011,2019-12-19 08:36:06,2019-12-19 09:00:48
7,16,2,mp4a,44100,159 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_013_2019-12-21 04-11-35.mp4,013,2019-12-21 04:11:35,2019-12-21 04:27:37
8,16,2,mp4a,44100,170 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_014_2019-12-20 07-13-25.mp4,014,2019-12-20 07:13:25,2019-12-20 07:34:47
9,16,2,mp4a,44100,163 Mbps,0,24,"[isom, iso2, avc1, mp41]",avc1,0000:00:00 00:00:00,...,0,0000:00:00 00:00:00,0.00%,23.976,72,72,OMV8_SVY_WA2019_V01_015_2019-12-20 05-59-36.mp4,015,2019-12-20 05:59:36,2019-12-20 06:16:58


In [21]:
videos =videos[['Station','StartTime','EndTime','Duration','BaseName']]

In [23]:
videos.to_csv(r'../outputs/hd_video.csv')

In [43]:
usbl = pd.read_csv('..\WA2019_V01\LOG\output.csv',parse_dates=['time'],index_col='time')

In [44]:
usbl.head()

,depth_meters,latitude,longitude,usbl_latitude,usbl_longitude
time,,,,,
2019-12-18 00:00:00,75.2,-19.225628,118.826020,NaN,NaN
2019-12-18 00:00:02,75.2,-19.225637,118.826010,-19.225637,118.826010
2019-12-18 00:00:04,75.2,-19.225645,118.825997,-19.225645,118.825997
2019-12-18 00:00:06,75.2,-19.225653,118.825990,-19.225653,118.825990
2019-12-18 00:00:08,75.0,-19.225662,118.825982,-19.225662,118.825982


In [57]:
for index,row in videos.iterrows():
    usbl['Station'] = row.Station
    tow = usbl[(usbl.index>=row.StartTime) & (usbl.index<=row.EndTime)]
    tow.to_csv('../outputs/'+row.BaseName.replace('mp4','csv'))

In [3]:
hdfiles = glob.glob(r"../WA*/**/*.mp4",recursive=True)
data = pd.DataFrame(hdfiles,columns=['FilePath'])
data['BaseName'] = data.FilePath.apply(os.path.basename)
data['Station']=data.BaseName.str.split('_',expand=True)[4]
data['FileNameTime']=data.BaseName.str.split('_',expand=True)[5].apply(extract_time)
data['ScreenTime'] =data['FileNameTime']
data.head()


,FilePath,BaseName,Station,FileNameTime,ScreenTime
0,..\WA2019_V01\OMV8\WA2019_V01_002\VIDEO\OMV8_S...,OMV8_SVY_WA2019_V01_002_2019-12-20 09-23-47.mp4,002,2019-12-20 09:23:47,2019-12-20 09:23:47
1,..\WA2019_V01\OMV8\WA2019_V01_003\VIDEO\OMV8_S...,OMV8_SVY_WA2019_V01_003_2019-12-21 09-06-45.mp4,003,2019-12-21 09:06:45,2019-12-21 09:06:45
2,..\WA2019_V01\OMV8\WA2019_V01_005\VIDEO\OMV8_S...,OMV8_SVY_WA2019_V01_005_2019-12-21 08-00-31.mp4,005,2019-12-21 08:00:31,2019-12-21 08:00:31
3,..\WA2019_V01\OMV8\WA2019_V01_006\VIDEO\OMV8_S...,OMV8_SVY_WA2019_V01_006_2019-12-20 08-22-35.mp4,006,2019-12-20 08:22:35,2019-12-20 08:22:35
4,..\WA2019_V01\OMV8\WA2019_V01_007\VIDEO\OMV8_S...,OMV8_SVY_WA2019_V01_007_2019-12-21 06-56-54.mp4,007,2019-12-21 06:56:54,2019-12-21 06:56:54


In [33]:
data.to_csv(r'../outputs/hd_input.csv')

In [58]:
row= videos.iloc[0]
tow = usbl[(usbl.index>=row.StartTime) & (usbl.index<=row.EndTime)]

In [59]:
tow.head()

,depth_meters,latitude,longitude,usbl_latitude,usbl_longitude,Station
time,,,,,,
2019-12-20 09:23:48,137.0,-18.689237,118.702137,-18.689237,118.702137,FPSPNW 62
2019-12-20 09:23:50,137.0,-18.689242,118.702130,-18.689242,118.702130,FPSPNW 62
2019-12-20 09:23:52,137.2,-18.689245,118.702123,-18.689245,118.702123,FPSPNW 62
2019-12-20 09:23:54,137.0,-18.689243,118.702113,-18.689243,118.702113,FPSPNW 62
2019-12-20 09:23:56,137.2,-18.689243,118.702100,-18.689243,118.702100,FPSPNW 62


In [60]:
max(tow.latitude - tow.usbl_latitude)


0.0

In [61]:
row

AudioBitsPerSample                                                   16
AudioChannels                                                         2
AudioFormat                                                        mp4a
AudioSampleRate                                                   44100
AvgBitrate                                                     161 Mbps
Balance                                                               0
BitDepth                                                             24
CompatibleBrands                               [isom, iso2, avc1, mp41]
CompressorID                                                       avc1
CreateDate                                          0000:00:00 00:00:00
CurrentTime                                                         0 s
Directory                                   ./OMV8/WA2019_V01_002/VIDEO
Duration                                                        0:19:44
Encoder                                                   Lavf58